In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

import base64

import crud
from crud import AnimalShelter

#Should Widen Jupyter Notebook Cell Width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))




###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Final Project')

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Start of layout
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    #Image
    html.Center(
        html.A([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
                     style={'height': '15%', 'width': '15%'})
        ], href='https://www.snhu.edu')),
    
    #Name Header
    html.Center(html.B(html.H1('John Elbogen'), style={'font=size': '12px'})),    
    html.Hr(),
    
    #Dropdown
    html.Div(
        style={'display': 'inline'},
        children=[
            dcc.Dropdown(
                id='my-dropdown',
                options=[
                    {'label': 'Water Rescue', 'value': '0'},
                    {'label': 'Mountain or Wilderness Rescue', 'value': '1'},
                    {'label': 'Disaster Rescue or Individual Tracking', 'value': '2'},
                    {'label': 'Reset', 'value': '3'},
                ],
                searchable=False
            )],
    ),
    html.Hr(),
    
    #Datatable
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],        
        data=df.to_dict('records'),
        
        editable=False,
        #filter_action=False,
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')], #Output('datatable-id','page_current')
              [Input('my-dropdown', 'value')]
             )
def update_dashboard(value):
    
    #Queries
    water_query = { 'breed': { '$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
    'sex_upon_outcome': 'Intact Female',
    'age_upon_outcome_in_weeks': { '$lte': 156},
    'age_upon_outcome_in_weeks': { '$gte': 26} }
    
    mountain_query = { 'breed': { '$in': ['German Shepherd', 'Alaskan Malamute', ' Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
    'sex_upon_outcome': 'Intact Male',
    'age_upon_outcome_in_weeks': { '$lte': 156},
    'age_upon_outcome_in_weeks': { '$gte': 26} }
    
    disaster_query = { 'breed': { '$in': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler']},
    'sex_upon_outcome': 'Intact Male',
    'age_upon_outcome_in_weeks': { '$lte': 300},
    'age_upon_outcome_in_weeks': { '$gte': 20} }
    
    if (value == '0'):
        df = pd.DataFrame.from_records(shelter.read(water_query))
    elif (value == '1'):
        df = pd.DataFrame.from_records(shelter.read(mountain_query))
    elif (value == '2'):
        df = pd.DataFrame.from_records(shelter.read(disaster_query))
    elif (value == '3'):
        df = pd.DataFrame.from_records(shelter.read({}))
        
    #sets the current page to zero to prevent bug, if user is on page X, and filters
    #less than X pages of results
    #page_current= 0    
    #Unfortunately this caused the pie graph to not update correctly and had to be removed!
    

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    return (data,columns)

    #return (data,columns,page_current)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(figure = px.pie(df, values=df.age_upon_outcome_in_weeks, names=df.breed, title='Combined Age By Breed'))
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '500px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]


app